In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import pandas as pd
from descriptors.dataset import MoleculesDataset, mol_collate_func
from torch.utils.data import DataLoader
from descriptastorus.descriptors.DescriptorGenerator import MakeGenerator
from tqdm import tqdm
from smiles_only_multilabel.data_utils import construct_dataset_multilabel, load_data_from_smiles, mol_collate_func
from sklearn.preprocessing import LabelBinarizer

In [4]:
data = pd.read_csv(
    '/home/dionizije/Documents/drug_attrition_oracle/data/chembl_drugbank_p4_21_7_+missing_labels+withdrawn_tox.csv'
)[['smiles', 'withdrawn_label', 'Toxicity type']]
data = data.sample(frac=1, random_state=0)

In [5]:
data

,smiles,withdrawn_label,Toxicity type
1854,N=S(=O)(O)Cc1noc2ccccc12,0,Safe
2620,O[C@@H]([C@H]1CC[C@@H](CC2=CC=C(NC(=O)[C@@H]3C...,0,Safe
252,OCCN1C[C@H](O)[C@@H](O)[C@H](O)[C@H]1CO,0,Safe
991,CC(C)OC(=O)CCC/C=C\C[C@H]1[C@@H](O)C[C@@H](O)[...,0,Safe
1739,CC(N)Cc1ccc(O)cc1,0,Safe
...,...,...,...
1731,CN1CCC(OC(c2ccccc2)c2ccccc2)CC1,0,Safe
763,C[C@]12C[C@H](O)[C@H]3[C@@H](CCC4=CC(=O)CC[C@@...,0,Safe
835,O=C(O)CCCO,0,Safe
1653,O=c1ccc2ccccc2o1,1,toxic_hepa


In [6]:
labels = LabelBinarizer().fit_transform(data['Toxicity type'])

In [27]:
test_data = data

In [8]:
labels[:1]

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [9]:
X_test, y_test = load_data_from_smiles(test_data['smiles'],
                                       test_data['withdrawn_label'],
                                       one_hot_formal_charge=True)

In [10]:
y2_test = labels[:1]

In [11]:
test_dataset = construct_dataset_multilabel(X_test, y_test, y2_test)

In [12]:
test_loader = DataLoader(test_dataset, num_workers=0, collate_fn=mol_collate_func, batch_size=1)

In [20]:
train_test_splitter = StratifiedKFold(n_splits=5)

In [19]:
from sklearn.model_selection import StratifiedKFold

In [25]:
for k, (train_index, test_index) in enumerate(
            train_test_splitter.split(data, data['withdrawn_label'], data['Toxicity type'])
    ):
    y2_test = labels[test_index]

In [24]:
len(y2_test)

544

In [32]:
from sklearn.utils.class_weight import compute_class_weight

In [40]:
list(data['Toxicity type'].unique()

array(['Safe', 'toxic_neuro', 'toxic_cardio', 'toxic_teratogen',
       'toxic_respiratory', 'toxic_systemic', 'toxic_gastro',
       'toxic_carcinogenic', 'toxic_hepa', 'toxic_dermis', 'toxic_hema',
       'toxic_opthalmic', 'toxic_nephro', 'toxic_muscoskelet',
       'carinogenicity', 'toxic_systemic; toxic_dermis',
       'toxic_systemic; toxic_gastro', 'toxic_hema; toxic_systemic'],
      dtype=object)

In [51]:
import torch

In [53]:
torch.Tensor(compute_class_weight(classes=data['Toxicity type'].unique(), y=data['Toxicity type'], class_weight='balanced')).size()

torch.Size([18])